<a href="https://colab.research.google.com/github/alokk2005/cnn-cat-vs-dog-classification/blob/main/cnn_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
img_size = (128, 128)
batch_size = 32

In [ ]:
(ds_train, ds_val), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

In [ ]:
def preprocess(img, label):
    img = tf.image.resize(img, img_size)
    img = img / 255.0
    return img, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = ds_train.map(preprocess).batch(batch_size).prefetch(AUTOTUNE)
val_ds = ds_val.map(preprocess).batch(batch_size).prefetch(AUTOTUNE)


In [ ]:

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

In [ ]:
test_loss, test_acc = model.evaluate(val_ds)
print("Validation Accuracy:", test_acc)

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.show()

In [ ]:
img = image.load_img("/content/rj4xrDA4vjAtkYrRB3xXK3pP5uHlbHc0O2Ye4DN9-AKh3NY9WBPGeGv4-JD3_v3KAGDuBrZrjMfnSoV-gju82XWN3_zmKng7rC-eLs8xbF8.jpg", target_size=(128,128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("Dog 🐶")
else:
    print("Cat 🐱")